In [1]:
# exports relevant environment variables
import os

# expects SOMOS dataset is in data/SOMOS in this directory
os.environ['SOMOS_COMBINED_SCORES_LOC'] = 'data/SOMOS/raw_scores_with_metadata/raw_scores.tsv'
os.environ['SOMOS_TRAIN_SCORES_LOC'] = 'data/SOMOS/raw_scores_with_metadata/split1/raw_scores_removed_excess_gt_trainset.tsv'
os.environ['SOMOS_VAL_SCORES_LOC'] = 'data/SOMOS/raw_scores_with_metadata/split1/raw_scores_removed_excess_gt_validset.tsv'
os.environ['SOMOS_TEST_SCORES_LOC'] = 'data/SOMOS/raw_scores_with_metadata/split1/raw_scores_removed_excess_gt_testset.tsv'
os.environ['SOMOS_AUDIO_DIR'] = 'data/SOMOS/audios'
# saving runs and models in the /runs and /pretrained directories, respectively
os.environ['RUNS_DIR'] = 'runs'
os.environ['SAVED_MODELS_DIR'] = 'pretrained'

In [3]:
# train mosnet model as baseline
from src.mosnet.train import train as train_mosnet

mosnet = train_mosnet('src/mosnet/config.yml', run_key='prod', early_stopping_min_epochs=100)

2023-11-11 16:47:57.904504 :: START RUN


KeyboardInterrupt: 